# LAB 5 MOS: Optimización Multiobjetivo

- Samuel Augusto Hernandez 202213772
- Tomas Alberto Rodriguez 2022128686

## Problema 1: Optimización Multiobjetivo en Distribución de Recursos para Misión Humanitaria

### Formulación del modelo Multiobjetivo

#### Conjuntos

#### Parámetros

#### Variables de decision

#### Funciones objetivo

#### Restricciones

#### Modelo de optimización


### Implementación del modelo

In [7]:
#codigo

#### Visualizacion del frente de Pareto

In [8]:
#codigo

### Analisis

## Problema 2: Optimización Multiobjetivo en Planificación de Rutas de Inspección

### Formulación del modelo Multiobjetivo

#### Conjuntos

#### Parámetros

#### Variables de decision

#### Funciones objetivo

#### Restricciones

### Modelo de optimización


### Implementación del modelo

In [9]:
#codigo

#### Visulizacion del frente de Pareto

In [10]:
#codigo

### Analisis